# Heuristic Optimization : Implementation exercise 1, results

This IPython notebook analyzes the behaviour of my implementation when using diferent parameters.

First, we import some usefull tools

In [1]:
import os
import subprocess
import time
import numpy as np
import scipy.stats
import itertools
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Then we define some functions needed to genrerate all the combinations of arguments we want to analyze plus some utilities to effectively call the binary

In [2]:
NEIGH = ["--transpose", "--exchange", "--insert", "--vnd-tei", "--vnd-tie"]
INIT = ["--random-init", "--srz"]
IMPROVE = ['--first', '--best']
def run_binary(t):
    instance, init, improvement, neighbourhood = t
    try:
        init = INIT[init]
        improvement = IMPROVE[improvement]
        neighbourhood = NEIGH[neighbourhood]
        command = "./fssp " + " ".join([instance, init, improvement, neighbourhood])
        comp_time = -time.time()
        output = subprocess.check_output(command, shell=True, stderr=subprocess.DEVNULL)
        comp_time = comp_time + time.time()
        score = output.decode('utf-8').strip()
        return command, int(score), comp_time
    except Exception as e:
        print(e)
        return command, None, None


def instance_calls(instance):
    for init in [0, 1]:
        for improvement in [0, 1]:
            for neighbourhood in [0, 1, 2, 3, 4]:
                if neighbourhood in (3,4) and (improvement == 1 or init == 0):
                    continue # skip --vnd-* combined with --random-init or --best
                    # as it will be ignored by the binary (see ./fssp -h)
                yield (instance, init, improvement, neighbourhood)
        
    

def get_instance_files():
    for filename in os.listdir("instances"):
        if filename != "bestSolutions.txt":
            yield os.path.join("instances/", filename)

In [3]:
import itertools
binary_calls = list(itertools.chain(*[instance_calls(name) for name in get_instance_files()]))

In [4]:
len(binary_calls)

840

After generating all the combinaisons, we can call out binary using `run_binary`. 
We encapsulate everything in a `Pool` for multiprocessing and `tqdm` to have a progess bar

In [5]:
import multiprocessing
from tqdm import tqdm as tqdm

In [6]:
p = multiprocessing.Pool(None) # use all cpus
res = list(tqdm(p.imap_unordered(run_binary, binary_calls), total=len(binary_calls)))

100%|██████████| 840/840 [06:21<00:00,  2.12s/it]


In [7]:
import pandas as pd

def ntype(s):
    for n in NEIGH:
        if n in s:
            return n

In the next few cells, we format the results to push them inside a Pandas DataFrame for easy manipulation.
Then we read the best solutions from a file and merge it with our DataFrame

In [8]:
processed = [
    {
        "instance": s.split()[1][10:],
        "n_jobs": int(s.split()[1][10:].split("_")[0]),
        "init": "srz" if "srz" in s else "random",
        "pivot": "best" if "best" in s else "first",
        "neighborhood": ntype(s).replace("--", ""),
        "score": score,
        "time": time,
        
    }
    for s, score, time in res
]

In [9]:
df = pd.DataFrame(processed)
df.sample()

,init,instance,n_jobs,neighborhood,pivot,score,time
420,random,50_20_05,50,transpose,best,857048,0.015512


In [10]:
best = pd.read_csv("instances/bestSolutions.txt")
best.columns = ["instance", "best_solution"]
best.instance = best.instance.str.strip()
best.best_solution = best.best_solution.astype(int)
best.sample()

,instance,best_solution
20,100_20_21,1709120


In [11]:
merged = pd.merge(df, best, on="instance", how="left")
merged.sample()

,init,instance,n_jobs,neighborhood,pivot,score,time,best_solution
382,srz,100_20_07,100,exchange,first,1831119,0.710617,1783580


We see that none of our solutions are better than the best known (that was expected)

In [12]:
merged[merged.best_solution >= merged.score]

,init,instance,n_jobs,neighborhood,pivot,score,time,best_solution


# Exercise 1

We filter the invocations to keep only what we need for the first exercise and add a new column : the relative deviation

In [13]:
ex1 = merged[(merged.neighborhood != "vnd-tei") & (merged.neighborhood != "vnd-tie")]

In [14]:
ex1['rel_deviation'] = 100 * (ex1['score'] - ex1['best_solution']) / ex1['best_solution']

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Mean deviation per method

In [44]:
ex1.groupby(["init", "pivot", "neighborhood"]).mean()[["rel_deviation"]]

rel_deviation
init   pivot neighborhood               
random best  exchange           4.526442
             insert             9.536986
             transpose         37.422758
       first exchange           1.840076
             insert             6.995218
             transpose         36.153227
srz    best  exchange           3.109973
             insert             3.161487
             transpose          4.144504
       first exchange           2.988648
             insert             2.935848
             transpose          4.130356

## Mean computation time per method and instance size

In [48]:
ex1.groupby(["init", "pivot", "neighborhood", "n_jobs"]).mean()[["time"]]

time
init   pivot neighborhood n_jobs           
random best  exchange     50       0.206074
                          100      3.410456
             insert       50       0.250388
                          100      3.855229
             transpose    50       0.017638
                          100      0.080919
       first exchange     50       0.645275
                          100     15.081069
             insert       50       0.830786
                          100     19.725573
             transpose    50       0.022015
                          100      0.083597
srz    best  exchange     50       0.057918
                          100      0.771066
             insert       50       0.057227
                          100      0.751073
             transpose    50       0.015344
                          100      0.042693
       first exchange     50       0.053892
                          100      1.070865
             insert       50       0.071624
                          100      1.092688
             transpose    50       0.016870
                          100      0.040687

## Statistical test preparation

We group every solution deviation resp. time by it's methods and store it in `deviation_per_algo` resp. `time_per_algo`

In [50]:
deviation_per_algo = {}
for keys, rows in ex1.groupby(["init", "pivot", "neighborhood"]):
    deviation_per_algo[keys] = rows.rel_deviation.as_matrix()

In [73]:
time_per_algo = {}
for keys, rows in ex1.groupby(["init", "pivot", "neighborhood"]):
    time_per_algo[keys] = rows.time.as_matrix()

## Comparaison of the initialization method

Smaller is best, `True` means that the p-value is lover than 0.05 thus there is a significant difference between the two.

In [64]:
compare_init = np.zeros((6,))
a = []
r = list(itertools.product(['first', 'best'], ['exchange', 'insert', 'transpose']))
for i, rest in enumerate(r):
    algo1 = ("random",) + rest
    algo2 = ("srz",) + rest
    t, p =  scipy.stats.ttest_rel(deviation_per_algo[algo1], deviation_per_algo[algo2])
    compare_init[i] = p
    a.append("random" if t < 0 else "srz")
compare_init = pd.DataFrame(compare_init < 0.05, columns=["random vs srz"])
compare_init["algo"] = list(map(lambda x: "%s-%s" % x, r))
compare_init["smaller"] = a
compare_init.set_index("algo", inplace=True)
compare_init

,random vs srz,smaller
algo,,
first-exchange,True,random
first-insert,True,srz
first-transpose,True,srz
best-exchange,True,srz
best-insert,True,srz
best-transpose,True,srz


## Comparaison of the pivot method

Smaller is best, `True` means that the p-value is lover than 0.05 thus there is a significant difference between the two.

### Deviation

In [63]:
compare_pivot = np.zeros((6,))
a = []
r = list(itertools.product(['random', 'srz'], ['exchange', 'insert', 'transpose']))
for i, rest in enumerate(r):
    algo1 = (rest[0], "first", rest[1])
    algo2 = (rest[0], "best", rest[1])
    t, p = scipy.stats.ttest_rel(deviation_per_algo[algo1], deviation_per_algo[algo2])
    compare_pivot[i] = p
    a.append("first" if t < 0 else "best")

compare_pivot = pd.DataFrame(compare_pivot < 0.05, columns=["first vs best"])
compare_pivot["algo"] = list(map(lambda x: "%s-%s" % x, r))
compare_pivot.set_index("algo", inplace=True)
compare_pivot["smaller"] = a
compare_pivot

,first vs best,smaller
algo,,
random-exchange,True,first
random-insert,True,first
random-transpose,True,first
srz-exchange,False,first
srz-insert,True,first
srz-transpose,False,first


### Computation time

In [74]:
compare_pivot = np.zeros((6,))
a = []
r = list(itertools.product(['random', 'srz'], ['exchange', 'insert', 'transpose']))
for i, rest in enumerate(r):
    algo1 = (rest[0], "first", rest[1])
    algo2 = (rest[0], "best", rest[1])
    t, p = scipy.stats.ttest_rel(time_per_algo[algo1], time_per_algo[algo2])
    compare_pivot[i] = p
    a.append("first" if t < 0 else "best")

compare_pivot = pd.DataFrame(compare_pivot < 0.05, columns=["first vs best"])
compare_pivot["algo"] = list(map(lambda x: "%s-%s" % x, r))
compare_pivot.set_index("algo", inplace=True)
compare_pivot["smaller"] = a
compare_pivot

,first vs best,smaller
algo,,
random-exchange,True,best
random-insert,True,best
random-transpose,False,best
srz-exchange,True,best
srz-insert,True,best
srz-transpose,False,first


## Comparaison of the neighborhood method

Smaller is best, `True` means that the p-value is lover than 0.05 thus there is a significant difference between the two.

### Deviation

In [72]:
compare_neigh = np.zeros((4,3))
a, b, c = [], [], []
r = list(itertools.product(['random', 'srz'], ['first', 'best']))
for i, rest in enumerate(r):
    algo1 = rest + ("exchange",)
    algo2 = rest + ("insert",)
    algo3 = rest + ("transpose",)
    t, p = scipy.stats.ttest_rel(deviation_per_algo[algo1], deviation_per_algo[algo2])
    compare_neigh[i][0]  = p
    a.append("exchange" if t < 0 else "insert")
    t, p = scipy.stats.ttest_rel(deviation_per_algo[algo1], deviation_per_algo[algo3])
    compare_neigh[i][1] = p
    b.append("exchange" if t < 0 else "transpose")
    t, p = scipy.stats.ttest_rel(deviation_per_algo[algo2], deviation_per_algo[algo3])
    compare_neigh[i][2] = p
    c.append("insert" if t < 0 else "transpose")

compare_neigh = pd.DataFrame(compare_neigh < 0.05, columns=["exchange vs insert", "exchange vs transpose", "insert vs transpose"])
compare_neigh["algo"] = list(map(lambda x: "%s-%s" % x, r))
compare_neigh.set_index("algo", inplace=True)
compare_neigh["smaller ei"] = a
compare_neigh["smaller et"] = b
compare_neigh["smaller it"] = c
compare_neigh

,exchange vs insert,exchange vs transpose,insert vs transpose,smaller ei,smaller et,smaller it
algo,,,,,,
random-first,True,True,True,exchange,exchange,insert
random-best,True,True,True,exchange,exchange,insert
srz-first,False,True,True,insert,exchange,insert
srz-best,False,True,True,exchange,exchange,insert


### Computation time

In [75]:
compare_neigh = np.zeros((4,3))
a, b, c = [], [], []
r = list(itertools.product(['random', 'srz'], ['first', 'best']))
for i, rest in enumerate(r):
    algo1 = rest + ("exchange",)
    algo2 = rest + ("insert",)
    algo3 = rest + ("transpose",)
    t, p = scipy.stats.ttest_rel(time_per_algo[algo1], time_per_algo[algo2])
    compare_neigh[i][0]  = p
    a.append("exchange" if t < 0 else "insert")
    t, p = scipy.stats.ttest_rel(time_per_algo[algo1], time_per_algo[algo3])
    compare_neigh[i][1] = p
    b.append("exchange" if t < 0 else "transpose")
    t, p = scipy.stats.ttest_rel(time_per_algo[algo2], time_per_algo[algo3])
    compare_neigh[i][2] = p
    c.append("insert" if t < 0 else "transpose")

compare_neigh = pd.DataFrame(compare_neigh < 0.05, columns=["exchange vs insert", "exchange vs transpose", "insert vs transpose"])
compare_neigh["algo"] = list(map(lambda x: "%s-%s" % x, r))
compare_neigh.set_index("algo", inplace=True)
compare_neigh["smaller ei"] = a
compare_neigh["smaller et"] = b
compare_neigh["smaller it"] = c
compare_neigh

,exchange vs insert,exchange vs transpose,insert vs transpose,smaller ei,smaller et,smaller it
algo,,,,,,
random-first,True,True,True,exchange,transpose,transpose
random-best,True,True,True,exchange,transpose,transpose
srz-first,False,True,True,exchange,transpose,transpose
srz-best,False,True,True,insert,transpose,transpose


# Exercise 2

We are now working on the VND part.

We filter the invocations to keep only what we need for the first exercise and add a new column : the relative deviation

In [22]:
ex2 = merged[(merged.neighborhood == "vnd-tei") | (merged.neighborhood == "vnd-tie")]

In [23]:
ex2['rel_deviation'] = 100 * (ex2['score'] - ex2['best_solution']) / ex2['best_solution']

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Mean of the relative deviation for each neighborhood

In [79]:
ex2.groupby(["neighborhood"]).mean()[["rel_deviation"]]

,rel_deviation
neighborhood,
vnd-tei,2.550408
vnd-tie,2.745331


## Mean of the computation time for each neighborhood

In [78]:
ex2.groupby(["neighborhood", "n_jobs"]).mean()[["time"]]

time
neighborhood n_jobs          
vnd-tei      50      0.073599
             100     1.154808
vnd-tie      50      0.064993
             100     0.728432

## Percentage of improvement against the single neighborhoods

In [106]:
ex2.groupby(["neighborhood"]).mean()[["rel_deviation"]].T

neighborhood,vnd-tei,vnd-tie
rel_deviation,2.550408,2.745331


In [107]:
ex1.groupby(["neighborhood"]).mean()[["rel_deviation"]].T[["exchange", "insert"]]

neighborhood,exchange,insert
rel_deviation,3.116285,5.657385


In [114]:
print("tei vs exchange: %f" % (3.116285/2.550408))
print("tei vs insert: %f" % (5.657385/2.550408))

print("tie vs exchange: %f" % (3.116285/2.745331))
print("tie vs insert: %f" % (5.657385/2.745331))

tei vs exchange: 1.221877
tei vs insert: 2.218227
tie vs exchange: 1.135122
tie vs insert: 2.060730


## T-test to compare the scores of both VND

In [115]:
deviation_per_algo = {}
for keys, rows in ex2.groupby(["init", "pivot", "neighborhood"]):
    deviation_per_algo[keys] = rows.rel_deviation.as_matrix()

In [118]:
t, p = scipy.stats.ttest_rel(deviation_per_algo[('srz', 'first', 'vnd-tie')], deviation_per_algo[('srz', 'first', 'vnd-tei')])

In [119]:
print(p, p < 0.05)

0.0350352305722 True


t is positive, "transpose, exchange, insert" is preferable

In [120]:
t

2.1576809268739678